In [195]:
import pandas as pd
import requests
import time
import numpy as np
import panel as pn
import json
pn.extension('tabulator')
import hvplot.pandas

Enter team tag below

In [196]:
team_name = "TSpirit"

Reading and processing of pro players list

In [197]:
players = pd.read_json('JSON_DATA/pro_players_list.json')

In [198]:
players = players.drop(['steamid', 'avatar', 'avatarmedium', 'avatarfull',
       'profileurl', 'personaname', 'last_login', 'full_history_time',
       'cheese', 'fh_unavailable', 'is_pro', 'team_id', 'loccountrycode'], axis=1)

Parsing and processing recent games of team

In [199]:

team_t = players[players.team_tag == team_name]
team_t.to_json('temp/team_temp.json', orient = 'records')
    
with open("temp/team_temp.json", "r") as read_file:
    team_t = json.load(read_file)
for player in team_t:
    check = False
    while check == False:
        try:
            accaunt_id = str(player['account_id'])
            url = 'https://api.opendota.com/api/players/' + accaunt_id+ '/matches'
            r = requests.get(url)
            with open('JSON_DATA/'+ accaunt_id + 'updated.json', 'wb') as outfile:
                outfile.write(r.content)
            time.sleep(0.5)
            check = True
        except: time.sleep(1)
heroes = pd.read_csv('CSV_DATA/heroes.csv')
matches = pd.read_csv('CSV_DATA/matches_table_empty.csv')
team_t = pd.read_json('temp/team_temp.json')
for value in team_t.account_id.values:
    match = pd.read_json('JSON_DATA/' + str(value) + 'updated.json')
    match = match.head(1000)
    match["side"] = (match["player_slot"]>100).astype(int)
    match["win"] = abs(match["side"]-match["radiant_win"])
    match["win1"] = match["win"]
    match.drop(columns = ['radiant_win','player_slot'], inplace = True)
    match.win = np.where(match.win == 0, -1, 1)
    match = match.sort_values(by='match_id').reset_index(drop = True)
    match['winsum'] = match["win"].cumsum()
    match["win"] = match["win1"]
    match['hero'] = match['hero_id'].map(heroes.set_index('id')['name'])
    match.drop(columns=['game_mode', 'lobby_type', 'version',
                        'leaver_status', 'party_size', 'skill', 'average_rank',
                        'hero_id', "win1"], inplace=True)
    match = match[['match_id', 'hero', 'duration', 'win','winsum', 'start_time', 'kills', 'deaths', 'assists', 'side']]
    match['player_name'] = team_t[team_t['account_id'] == value].name.values[0]
    matches = pd.concat([matches, match])
matches.to_csv('temp/matches_temp.csv', index=False)


Creating of dashboard's widgets

In [200]:
team = pd.read_json("temp/team_temp.json")

In [201]:
matches_temp = pd.read_csv('temp/matches_temp.csv')

In [202]:
team_names = players[players.team_tag == team_name]['name'].values.tolist()

Buttons for choosing of players

In [203]:
idf2 = matches_temp.interactive()
player = pn.widgets.RadioButtonGroup(
    name='Radio Button Group', options = team_names, button_type = 'success', sizing_mode = 'stretch_width')

Slider for choosing numer of games

In [204]:
games_slider = pn.widgets.IntSlider(name='Games slider', start=10, end=1000, step=10, value=500)

BokehModel(combine_events=True, render_bundle={'docs_json': {'b1f6639d-8c49-4171-9221-1033dd08b1a5': {'defs': …

IntSlider(end=1000, name='Games slider', start=10, step=10, value=500)

Matches with player's game stats

In [205]:
matches_pipline = idf2[idf2.player_name==player].tail(games_slider).reset_index(drop = True)

In [206]:
matches_table = matches_pipline.pipe(pn.widgets.Tabulator, pagination='local', 
                                     page_size = 10, sizing_mode='stretch_width') 

Winrate plot

In [207]:
winrate_pipline = idf2[idf2.player_name==player].tail(games_slider).sort_values(by='match_id').reset_index(drop = True)

In [208]:
winrate = winrate_pipline.hvplot(x = 'index', y='winsum',line_width=2, title="")

Heroes stats

In [210]:
heroes_pipline =  idf2[idf2.player_name==player].tail(games_slider).groupby(['hero']).agg({'hero':'size', 'win':'mean',
                                                                                   'kills':'mean', 'deaths':'mean', 'assists':'mean'}).rename(
                                                                                    columns={'hero':'count', 'win': 'winrate'}).sort_values(by='count', ascending = False)  

In [211]:
heroes_table = heroes_pipline.pipe(pn.widgets.Tabulator, pagination='local', 
                                     page_size = 10, sizing_mode='stretch_width')

In [213]:
text_bigtable = pn.widgets.StaticText(value='Last games stats')

Template for dashboard

In [214]:
template = pn.template.FastListTemplate(
    title='Dota2 pro-players dashboard', 
    sidebar=[pn.pane.Markdown(f"# {team_name}"), 
             pn.pane.PNG('dota2_social.png', sizing_mode='scale_both'),
             pn.pane.Markdown("#### This dashboard complements draft analytics. It allows you to evaluate the player's skill on various heroes and performance in recent games."), 
             pn.pane.Markdown("## Settings"),   
             games_slider],
    main=[ pn.Row(pn.Column(player,pn.pane.Markdown("### Last games stats"), matches_table.panel(width=1225))), 
           pn.Row(pn.Column(pn.pane.Markdown("### Winrate"),winrate.panel(width=600), margin=(0,25)),pn.Column(pn.pane.Markdown("### Heroes stats"), heroes_table.panel(width=600)))],
    accent_base_color="#4285F4",
    header_background="#cf262b",
)
template.show();

Launching server at http://localhost:43103
